### Binomial Tree Option Pricing

Formula explicited in the paper "Implementing Option Pricing Models using Python and Cython" by Sanjiv Das and Brian Granger.

It uses the Jarrow-Rudd model. It is oth widely known by students in finance and numerically intensive, requiring backward recursion on a tree.

In [1]:
import numpy as np
import math

In [2]:
def jarrow_rudd(initial_stock_price, strike_price, expiration_time, volatility,
                risk_free_rate, option_type, option_exercise=False, binomial_steps=10):
    """
    Implementation of the Jarrow-Rudd model to price option using a binomial tree.
    <param :option_type:> Integer ; 1 for Call and -1 for Put option
    <param :option_exercise:> Boolean ; True: American Option, False: European Option
    """
    time_step = expiration_time/binomial_steps
    
    # option price calculated at end of time_step based on up or down movement
    up_value = math.exp((risk_free_rate-0.5*math.pow(volatility,2))*time_step+volatility*math.sqrt(time_step))
    down_value = math.exp((risk_free_rate-0.5*math.pow(volatility,2))*time_step-volatility*math.sqrt(time_step))
    
    drift = math.exp(risk_free_rate*time_step)
    probability = (drift - down_value)/(up_value-down_value)
    
    # initializes the step table for the underlying stock value with numpy
    stock_value = np.zeros((binomial_steps+1,binomial_steps+1))
    # initializes the step table for the option value with numpy
    option_value = np.zeros((binomial_steps+1,binomial_steps+1))
    
    # initializes the starting value of the step table
    stock_value[0,0]=initial_stock_price
    
    # calculates the up and down movements of the underlying stock price step table
    for row in range(1,binomial_steps+1):
        stock_value[row,0] = stock_value[row-1,0]*up_value
        for column in range(1,row+1):
            stock_value[row,column]= stock_value[row-1,column-1]*down_value
    
    # apply the option function to the underlying stock price step table
    for column in range(binomial_steps+1):
        option_value[binomial_steps,column] = max(0, option_type * 
                                                  (stock_value[binomial_steps,column]-strike_price))
    # backward calculation of option price
    for row in range(binomial_steps-1,-1,-1):
        for column in range(row+1):
            option_value[row,column] = (probability*option_value[row+1,column]+
                                        (1-probability)*option_value[row+1,column+1])/drift
            if option_exercise:
                option_value[row,column] = max(option_value[row,column],option_type * 
                                               (stock_value[row,column]-strike_price))
    # returns the earliest calculated value of the option price
    return option_value[0,0]

In [3]:
jarrow_rudd(100.0,100.0,1.0,0.3,0.03,1,False, 100)

13.296923134168859